In [7]:
# Install the following libraries
# %pip install sepaxml

In [8]:
# Import libraries
import json
import sys
sys.path.append("..")
from utils.filesystem import *

# Import SEPA classes
from sepaxml import SepaDD

In [9]:
# Config file path
config_file = '../config.json'

# Load the config file
with open(config_file) as f:
    config = json.load(f)

# Load the basic config parameters
input_folder_remesa = config['input_folder_remesa']
input_file_remesa = config['input_file_remesa']

In [10]:
# Ignore UserWarning messages
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load a specific worksheet from the Excel file
try:
    dataframe = load_csv_to_dataframe (input_folder_remesa, input_file_remesa, 0)

    # Count the number of rows and columns
    rows, columns = dataframe.shape
    print(f"The dataframe has {rows} rows and {columns} columns")
except Exception as e:
    print(f"An error occurred: {e}")

The dataframe has 896 rows and 12 columns


In [11]:
# Remove rows that have ContactType__c = "Baja"
try:
    dataframe = dataframe[dataframe['ContactType__c'] != 'Baja']
    rows, columns = dataframe.shape
    print(f"The dataframe has {rows} rows and {columns} columns")
except Exception as e:
    print(f"An error occurred: {e}")

# Print the column names
print(dataframe.columns)

The dataframe has 660 rows and 12 columns
Index(['attributes', 'Id', 'ContactType__c', 'FirstName', 'MiddleName',
       'LastName', 'Name', 'Email', 'aesr_CuentaBancaria_NombreTitular__c',
       'aesr_CuentaBancaria_NombreBanco__c', 'aesr_CuentaBancaria_IBAN__c',
       'aesrCuotaSocio__c'],
      dtype='object')


In [13]:
import pandas as pd
from sepaxml import SepaDD
from datetime import datetime

df = dataframe

# Initialize the SEPA Direct Debit
sepa_dd = SepaDD(
    creditor={
        'name': "Your Company Name",
        'account': "YOUR_IBAN_HERE",
        'agent': "YOUR_BIC_HERE",
        'id': "YOUR_CREDITOR_ID_HERE"
    },
    payments=[
        {
            'id': 'PmtInf-' + datetime.now().strftime('%Y%m%d%H%M%S'),
            'due_date': datetime.now().strftime('%Y-%m-%d'),
            'type': 'OOFF',
            'transactions': []
        }
    ],
    batch=False  # Set to True if you want to process as a batch
)

# Adding transactions from the dataframe
for _, row in df.iterrows():
    transaction = {
        'name': row['aesr_CuentaBancaria_NombreTitular__c'],
        'iban': row['aesr_CuentaBancaria_IBAN__c'],
        'bic': "",  # Optional: Add BIC if available
        'amount': row['aesrCuotaSocio__c'],
        'type': 'OOFF',  # One-Off Direct Debit
        'collection_date': datetime.now().strftime('%Y-%m-%d'),
        'mandate_id': 'MANDATE_' + row['Id'],
        'mandate_date': datetime.now().strftime('%Y-%m-%d'),
        'description': 'Membership Fee'
    }
    sepa_dd.add_payment_transaction(0, transaction)

# Generate the XML file
sepa_dd.export("sepa_dd.xml")


TypeError: SepaDD.__init__() got an unexpected keyword argument 'creditor_name'

In [12]:
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

df = dataframe

def create_sepa_xml(df):
    root = ET.Element('Document', xmlns="urn:iso:std:iso:20022:tech:xsd:pain.008.001.02")

    CstmrDrctDbtInitn = ET.SubElement(root, 'CstmrDrctDbtInitn')

    GrpHdr = ET.SubElement(CstmrDrctDbtInitn, 'GrpHdr')
    MsgId = ET.SubElement(GrpHdr, 'MsgId')
    MsgId.text = 'SEPA-DD-' + datetime.now().strftime('%Y%m%d%H%M%S')
    CreDtTm = ET.SubElement(GrpHdr, 'CreDtTm')
    CreDtTm.text = datetime.now().isoformat()
    NbOfTxs = ET.SubElement(GrpHdr, 'NbOfTxs')
    NbOfTxs.text = str(len(df))
    CtrlSum = ET.SubElement(GrpHdr, 'CtrlSum')
    CtrlSum.text = str(df['aesrCuotaSocio__c'].sum())

    InitgPty = ET.SubElement(GrpHdr, 'InitgPty')
    Nm = ET.SubElement(InitgPty, 'Nm')
    Nm.text = 'Your Company Name'

    PmtInf = ET.SubElement(CstmrDrctDbtInitn, 'PmtInf')
    PmtInfId = ET.SubElement(PmtInf, 'PmtInfId')
    PmtInfId.text = 'PmtInf-' + datetime.now().strftime('%Y%m%d%H%M%S')
    PmtMtd = ET.SubElement(PmtInf, 'PmtMtd')
    PmtMtd.text = 'DD'
    BtchBookg = ET.SubElement(PmtInf, 'BtchBookg')
    BtchBookg.text = 'true'
    NbOfTxs = ET.SubElement(PmtInf, 'NbOfTxs')
    NbOfTxs.text = str(len(df))
    CtrlSum = ET.SubElement(PmtInf, 'CtrlSum')
    CtrlSum.text = str(df['aesrCuotaSocio__c'].sum())

    PmtTpInf = ET.SubElement(PmtInf, 'PmtTpInf')
    SvcLvl = ET.SubElement(PmtTpInf, 'SvcLvl')
    Cd = ET.SubElement(SvcLvl, 'Cd')
    Cd.text = 'SEPA'
    LclInstrm = ET.SubElement(PmtTpInf, 'LclInstrm')
    Cd = ET.SubElement(LclInstrm, 'Cd')
    Cd.text = 'CORE'
    SeqTp = ET.SubElement(PmtTpInf, 'SeqTp')
    SeqTp.text = 'OOFF'

    ReqdColltnDt = ET.SubElement(PmtInf, 'ReqdColltnDt')
    ReqdColltnDt.text = datetime.now().strftime('%Y-%m-%d')

    Cdtr = ET.SubElement(PmtInf, 'Cdtr')
    Nm = ET.SubElement(Cdtr, 'Nm')
    Nm.text = 'Your Company Name'

    CdtrAcct = ET.SubElement(PmtInf, 'CdtrAcct')
    Id = ET.SubElement(CdtrAcct, 'Id')
    IBAN = ET.SubElement(Id, 'IBAN')
    IBAN.text = 'YOUR_IBAN_HERE'

    CdtrAgt = ET.SubElement(PmtInf, 'CdtrAgt')
    FinInstnId = ET.SubElement(CdtrAgt, 'FinInstnId')
    BIC = ET.SubElement(FinInstnId, 'BIC')
    BIC.text = 'YOUR_BIC_HERE'

    ChrgBr = ET.SubElement(PmtInf, 'ChrgBr')
    ChrgBr.text = 'SLEV'

    CdtrSchmeId = ET.SubElement(PmtInf, 'CdtrSchmeId')
    Id = ET.SubElement(CdtrSchmeId, 'Id')
    PrvtId = ET.SubElement(Id, 'PrvtId')
    Othr = ET.SubElement(PrvtId, 'Othr')
    Id = ET.SubElement(Othr, 'Id')
    Id.text = 'YOUR_CREDITOR_ID_HERE'
    SchmeNm = ET.SubElement(Othr, 'SchmeNm')
    Prtry = ET.SubElement(SchmeNm, 'Prtry')
    Prtry.text = 'SEPA'

    for index, row in df.iterrows():
        DrctDbtTxInf = ET.SubElement(PmtInf, 'DrctDbtTxInf')

        PmtId = ET.SubElement(DrctDbtTxInf, 'PmtId')
        EndToEndId = ET.SubElement(PmtId, 'EndToEndId')
        EndToEndId.text = row['Id']

        InstdAmt = ET.SubElement(DrctDbtTxInf, 'InstdAmt', Ccy='EUR')
        InstdAmt.text = str(row['aesrCuotaSocio__c'])

        DrctDbtTx = ET.SubElement(DrctDbtTxInf, 'DrctDbtTx')
        MndtRltdInf = ET.SubElement(DrctDbtTx, 'MndtRltdInf')
        MndtId = ET.SubElement(MndtRltdInf, 'MndtId')
        MndtId.text = 'MANDATE_' + row['Id']
        DtOfSgntr = ET.SubElement(MndtRltdInf, 'DtOfSgntr')
        DtOfSgntr.text = datetime.now().strftime('%Y-%m-%d')

        DbtrAgt = ET.SubElement(DrctDbtTxInf, 'DbtrAgt')
        FinInstnId = ET.SubElement(DbtrAgt, 'FinInstnId')

        Dbtr = ET.SubElement(DrctDbtTxInf, 'Dbtr')
        Nm = ET.SubElement(Dbtr, 'Nm')
        Nm.text = row['aesr_CuentaBancaria_NombreTitular__c']

        DbtrAcct = ET.SubElement(DrctDbtTxInf, 'DbtrAcct')
        Id = ET.SubElement(DbtrAcct, 'Id')
        IBAN = ET.SubElement(Id, 'IBAN')
        IBAN.text = row['aesr_CuentaBancaria_IBAN__c']

        RmtInf = ET.SubElement(DrctDbtTxInf, 'RmtInf')
        Ustrd = ET.SubElement(RmtInf, 'Ustrd')
        Ustrd.text = 'Membership Fee'

    tree = ET.ElementTree(root)
    tree.write('sepa_dd.xml', encoding='utf-8', xml_declaration=True)

# Generate the SEPA XML file
create_sepa_xml(df)


TypeError: cannot serialize nan (type float)